In [1]:
# import argparse 
# import sys
# sys.path.append('./grover/')
# import grover.util.parsing
# import grover.model.models
# import task.train

# ## Use pretrain config
# parser = argparse.ArgumentParser()
# subparser = parser.add_subparsers(title="subcommands",
#                                     dest="parser_name",
#                                     help="Subcommands for finetune, prediction, and fingerprint.")
# parser_finetune = subparser.add_parser('finetune', help="Fine tune the pre-trained model.")
# grover.util.parsing.add_finetune_args(parser_finetune)
# parser_eval = subparser.add_parser('eval', help="Evaluate the results of the pre-trained model.")
# grover.util.parsing.add_finetune_args(parser_eval)
# parser_predict = subparser.add_parser('predict', help="Predict results from fine tuned model.")
# grover.util.parsing.add_predict_args(parser_predict)
# parser_fp = subparser.add_parser('fingerprint', help="Get the fingerprints of SMILES.")
# grover.util.parsing.add_fingerprint_args(parser_fp)
# parser_pretrain = subparser.add_parser('pretrain', help="Pretrain with unlabelled SMILES.")
# grover.util.parsing.add_pretrain_args(parser_pretrain)

# grover_args = parser.parse_args("finetune --data_path grover/exampledata/finetune/bbbp.csv \
#                         --features_path grover/exampledata/finetune/bbbp.npz \
#                         --save_dir grover/model/finetune/bbbp/ \
#                         --checkpoint_path grover/model/tryout/model.ep3 \
#                         --dataset_type classification \
#                         --split_type scaffold_balanced \
#                         --ensemble_size 1 \
#                         --num_folds 3 \
#                         --no_features_scaling \
#                         --ffn_hidden_size 200 \
#                         --batch_size 32 \
#                         --epochs 10 \
#                         --init_lr 0.00015 \
#                         --no_cuda".split())

# grover.util.parsing.modify_train_args(grover_args)
# # train_args = grover.util.parsing.get_newest_train_args()
# features_scaler, scaler, shared_dict, test_data, train_data, val_data = task.train.load_data(grover_args, print, None)
# grover_model = grover.util.utils.load_checkpoint("./grover/grover_large.pt", current_args=grover_args, logger=None)



In [2]:
# import grover.data
# import torch

# grover_linear_layer = torch.nn.Linear(1200, 768)
# grover_layer = grover.model.layers.GTransEncoder(args=grover_args,
#                                           hidden_size=768,
#                                           edge_fdim=768,
#                                           node_fdim=768,
#                                           dropout=grover_args.dropout,
#                                           activation=grover_args.activation,
#                                           num_mt_block=1,
#                                           num_attn_head=grover_args.num_attn_head,
#                                           atom_emb_output="atom",
#                                           bias=grover_args.bias,
#                                           cuda=grover_args.cuda)

# mol_collator = grover.data.MolCollator(shared_dict=shared_dict, args=grover_args)

# num_workers = 4
# mol_loader = torch.utils.data.DataLoader(train_data, batch_size=grover_args.batch_size, shuffle=True,
#                     num_workers=num_workers, collate_fn=mol_collator)

# grover_model.train()
# for item in mol_loader:
#     _, batch, features_batch, mask, targets = item
#     f_atoms, f_bonds, a2b, b2a, b2revb, a_scope, b_scope, a2a = batch
#     # if next(model.parameters()).is_cuda:
#     #     mask, targets = mask.cuda(), targets.cuda()

#     # Run model
#     grover_model.zero_grad()
    
#     output = grover_model.grover((f_atoms, f_bonds, a2b, b2a, b2revb, a_scope, b_scope, a2a))

#     f_atoms, f_bonds = output['atom_from_atom'], output['bond_from_atom']
#     f_atoms = grover_linear_layer(f_atoms)
#     f_bonds = grover_linear_layer(f_bonds)
#     print(f_atoms.shape, f_bonds.shape)
    
#     output = grover_layer((f_atoms, f_bonds, a2b, b2a, b2revb, a_scope, b_scope, a2a))
#     f_atoms = output[0]
#     print(f_atoms.shape, f_bonds.shape)

#     output = grover_layer((f_atoms, f_bonds, a2b, b2a, b2revb, a_scope, b_scope, a2a))
#     f_atoms = output[0]
#     print(f_atoms.shape, f_bonds.shape)

#     break

In [8]:
### ========== retrieval metrics ==========
import numpy as np
import sklearn.metrics

def get_ranks(embedding1, embedding2):
    # assumes that the true value is diagonal
    ranks_tmp = []
    emb = sklearn.metrics.pairwise.cosine_similarity(embedding1, embedding2) # shape: (len(embedding1), len(embedding2))
    for k in range(emb.shape[0]):
        cid_locs = np.argsort(emb[k])[::-1] #sort high-to-low each column
        ranks = np.argsort(cid_locs) # get rank (original array order, but with rank instead of value)
        ranks_tmp.append(ranks[k] + 1)
    return np.array(ranks_tmp)

def print_ranks(ranks):
    print("Mean rank: {:.2f}".format(np.mean(ranks)))
    print("Hits at 1:", np.mean(ranks <= 1))
    print("Hits at 10:", np.mean(ranks <= 10))
    print("Hits at 20:", np.mean(ranks <= 20))
    print("Hits at 100:", np.mean(ranks <= 100))
    print("Hits at 500:", np.mean(ranks <= 500))
    print("Hits at 1000:", np.mean(ranks <= 1000))
    print("MRR:", np.mean(1/ranks))

# np.random.seed(seed=0)
# e2 = np.random.random(size=(1000,768))
# e3 = np.random.random(size=(1000,768))
# # for i in range(len(e2)):
# #     e2[i] += e3[i]*.1 # correlate text and mol embeddings

test_output = np.load('test_output_no_ca.npy', allow_pickle=True)
# test_output = np.load('test_output.npy', allow_pickle=True)
print(test_output.shape)
e1_mol = np.concatenate(test_output[:,0])
e1_text = np.concatenate(test_output[:,1])
# e2_text = np.concatenate(test_output[:,2])
# e3_mol = np.concatenate(test_output[:,3])

# retrieve mol from text
print("retrieving mol from text")
ranks_tmp = get_ranks(e1_text, e1_mol)
print_ranks(ranks_tmp)

# retrieve text from mol
print("retrieving text from mol")
ranks_tmp = get_ranks(e1_mol, e1_text)
print_ranks(ranks_tmp)



(413, 2)
retrieving mol from text
Mean rank: 6.67
Hits at 1: 0.3332323538321721
Hits at 10: 0.8588306573765525
Hits at 20: 0.940018176310209
Hits at 100: 0.9957588609512269
Hits at 500: 0.9996970614965162
Hits at 1000: 1.0
MRR: 0.5042599363284074
retrieving text from mol
Mean rank: 10.23
Hits at 1: 0.33414116934262345
Hits at 10: 0.8427749166919115
Hits at 20: 0.9212359890942139
Hits at 100: 0.9881853983641321
Hits at 500: 0.9975764919721296
Hits at 1000: 0.9987882459860649
MRR: 0.5054617838749837


In [4]:
test_output[0,0].shape

(8, 768)

In [6]:
# import numpy as np

# use_ca = False
# u_list = np.load('link_pred_ca={}_u_list.npy'.format(str(use_ca)))
# v_list = np.load('link_pred_ca={}_v_list.npy'.format(str(use_ca)))
# labels = np.load('link_pred_ca=True_labels.npy')

# u_list = u_list[labels==1]
# v_list = v_list[labels==1]
# u_list = u_list[:1000] 
# v_list = v_list[:1000]
# print(u_list.shape, v_list.shape, labels.shape)

# ranks_tmp = get_ranks(u_list, v_list)
# print_ranks(ranks_tmp)


In [7]:
# import pandas as pd

# d = pd.read_csv('chebi20.csv', sep='\t')
# print(d.columns)
# d['SMILES']